# Title by Owner

## Imports

In [26]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from datetime import datetime
from os import path
from utils.callbacks import SaveBestModelInMemory
from utils.submission import create_submission_zip

## Constants

In [27]:
NUM_CLASSES = 12
RANDOM_STATE = 42 # Seed for rng to make everything reproducible and deterministic af
SAVED_MODELS_PATH = "saved-models"
TENSORBOARD_LOGS_PATH = "tensorboard-logs"
SUBMISSIONS_PATH = "../submissions"

## Parameters

In [34]:
BATCH_SIZE = 64 # Number of samples in a mini batch
EPOCHS = 50 # Number of training epochs before the training is stopped
TEST_SPLIT = 0.15 # Percent of data to use for validation/testing

## Data Loading and Preprocessing

In [29]:
data = np.load(file="../dataset/x_train.npy")
labels = np.load(file="../dataset/y_train.npy")

def augment_sample(sample: np.ndarray) -> np.ndarray:
    noise = np.random.normal(0, 1, (36, 6))
    return sample + noise

min_samples_per_class = 250
total_added_samples = 0
for cls in range(NUM_CLASSES):
    class_data = data[labels == cls]
    samples_to_add = min_samples_per_class - len(class_data)
    if samples_to_add <= 0:
        print(f"Class {cls} has {len(class_data)} samples. Skipping.")
        continue

    print(f"Class {cls} has {len(class_data)} samples. Adding {samples_to_add} samples.")
    total_added_samples += samples_to_add
    augmented_data = []
    for _ in range(samples_to_add):
        sample = class_data[np.random.randint(0, len(class_data))]
        augmented_sample = augment_sample(sample)
        augmented_data.append(sample)
    data = np.concatenate([data, augmented_data])
    labels = np.concatenate([labels, [cls] * samples_to_add])

print(f"Added {total_added_samples} to the dataset, now all classes have at least {min_samples_per_class} samples")

categorical_labels = tf.keras.utils.to_categorical(labels, num_classes=NUM_CLASSES)
train_data, test_data, train_labels, test_labels = train_test_split(data, categorical_labels, test_size=TEST_SPLIT, random_state=RANDOM_STATE)

# Make sure everything was loaded correctly:
print(f"All samples shape: {data.shape}, all labels shape: {labels.shape}")
print(f"Train samples shape: {train_data.shape}, Train labels shape: {train_labels.shape}")
print(f"Test samples shape: {test_data.shape}, Test labels shape: {test_labels.shape}")
print("Samples per class:")
print("Total: ", np.sum(categorical_labels, axis=0))
print("Train: ", np.sum(train_labels, axis=0))
print("Test: ", np.sum(test_labels, axis=0))


Class 0 has 34 samples. Adding 216 samples.
Class 1 has 123 samples. Adding 127 samples.
Class 2 has 270 samples. Skipping.
Class 3 has 381 samples. Skipping.
Class 4 has 62 samples. Adding 188 samples.
Class 5 has 153 samples. Adding 97 samples.
Class 6 has 313 samples. Skipping.
Class 7 has 68 samples. Adding 182 samples.
Class 8 has 120 samples. Adding 130 samples.
Class 9 has 777 samples. Skipping.
Class 10 has 77 samples. Adding 173 samples.
Class 11 has 51 samples. Adding 199 samples.
Added 1312 to the dataset, now all classes have at least 250 samples
All samples shape: (3741, 36, 6), all labels shape: (3741,)
Train samples shape: (3179, 36, 6), Train labels shape: (3179, 12)
Test samples shape: (562, 36, 6), Test labels shape: (562, 12)
Samples per class:
Total:  [250. 250. 270. 381. 250. 250. 313. 250. 250. 777. 250. 250.]
Train:  [213. 215. 215. 331. 215. 213. 271. 213. 211. 666. 208. 208.]
Test:  [ 37.  35.  55.  50.  35.  37.  42.  37.  39. 111.  42.  42.]


## Model Definition

In [30]:
# Create the model inside the function this is a dummy implementation
def build_model(name: str, input_shape: tuple[int,...], classes: int) -> tf.keras.Model:
    # Build the neural network layer by layer
    input_layer = tf.keras.layers.Input(shape=input_shape, name="Input")

    # Feature extractor
    lstm = tf.keras.layers.LSTM(128, return_sequences=True)(input_layer)
    lstm = tf.keras.layers.LSTM(128)(lstm)
    dropout = tf.keras.layers.Dropout(.5, seed=RANDOM_STATE)(lstm)

    # Classifier
    classifier = tf.keras.layers.Dense(128, activation="relu")(dropout)
    output_layer = tf.keras.layers.Dense(classes, activation="softmax")(classifier)

    # Connect input and output through the Model class
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer, name=name)

    # Compile the model
    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics="accuracy")

    # Return the model
    return model

## Training

In [35]:
input_shape = train_data.shape[1:]
classes = NUM_CLASSES
model_name = "awesome-model-balanced-classes" # Give your model an awesome name for a 2% percent accuracy increase.

model = build_model(model_name, input_shape, classes)
model.summary()

run_id = datetime.utcnow().strftime("%Y-%m-%d-%H-%M-%S")
current_tensorboard_log_dir = f"{TENSORBOARD_LOGS_PATH}/{model_name}/{run_id}"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=current_tensorboard_log_dir)
print(f"Run tensorboard in a separate process with:\n"
      f"tensorboard --logdir {path.abspath(TENSORBOARD_LOGS_PATH)}\nor\n"
      f"tensorboard --logdir {path.abspath(current_tensorboard_log_dir)}")

best_weights_callback = SaveBestModelInMemory(metric="val_loss")

model.fit(x=train_data, y=train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(test_data, test_labels), callbacks=[tensorboard_callback, best_weights_callback])

Model: "awesome-model-balanced-classes"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 36, 6)]           0         
                                                                 
 lstm_2 (LSTM)               (None, 36, 128)           69120     
                                                                 
 lstm_3 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               16512     
                                                                 
 dense_3 (Dense)             (None, 12)                1548      
                                                                 
Total params: 218,764
Trainable para

50/50 [==============================] - 6s 79ms/step - loss: 1.9978 - accuracy: 0.3190 - val_loss: 1.5463 - val_accuracy: 0.4609
Epoch 2/50
50/50 [==============================] - 3s 67ms/step - loss: 1.3907 - accuracy: 0.5005 - val_loss: 1.1687 - val_accuracy: 0.5676
Epoch 3/50
50/50 [==============================] - 3s 67ms/step - loss: 1.1082 - accuracy: 0.5986 - val_loss: 1.0163 - val_accuracy: 0.6246
Epoch 4/50
50/50 [==============================] - 4s 71ms/step - loss: 0.9159 - accuracy: 0.6678 - val_loss: 0.9004 - val_accuracy: 0.6690
Epoch 5/50
50/50 [==============================] - 4s 74ms/step - loss: 0.8021 - accuracy: 0.7065 - val_loss: 0.8679 - val_accuracy: 0.6957
Epoch 6/50
50/50 [==============================] - 4s 75ms/step - loss: 0.7077 - accuracy: 0.7395 - val_loss: 0.8260 - val_accuracy: 0.7117
Epoch 7/50
50/50 [==============================] - 4s 77ms/step - loss: 0.6234 - accuracy: 0.7763 - val_loss: 0.8043 - val_accuracy: 0.7224
Epoch 8/50
50/50 [======

## Optional: Save model in memory

In [36]:
model.set_weights(best_weights_callback.best_weights)
saved_model_path = f"{SAVED_MODELS_PATH}/{model_name}/{run_id}"
model.save(saved_model_path)

INFO:tensorflow:Assets written to: saved-models/awesome-model-balanced-classes/2022-12-15-12-01-39/assets


INFO:tensorflow:Assets written to: saved-models/awesome-model-balanced-classes/2022-12-15-12-01-39/assets


## Optional: Create submission ZIP

In [37]:
submission_path = f"{SUBMISSIONS_PATH}/{model_name}/{run_id}"
create_submission_zip(submission_path, saved_model_path)

print(f"Created submission: {submission_path}.zip")

Created submission: ../submissions/awesome-model-balanced-classes/2022-12-15-12-01-39.zip


In [41]:
set1_score= 0.609431680774
classes_scores = [0.499999999947, 0.597701149374, 0.886486486436, 0.384615384566, 0.140350877146, 0.267857142807, 0.758293838812, 0.255319148885, 0.964705882301, 0.665271966477, 0.839999999947, 0.555555555502]

avg = sum(classes_scores) / len(classes_scores)

set1_prev = 0.6614
classes_scores_prev = [0.2857, 0.6098, 	0.8663,	0.4897,	0.0000, 0.0656,	0.7633, 0.0000,	0.8696,	0.7279,	0.8667,	0.4828, ]

avg_prev = sum(classes_scores_prev) / len(classes_scores_prev)

print(avg, avg_prev, len(classes_scores), len(classes_scores_prev))

0.5680131193500001 0.5022833333333333 12 12
